## Introduction

* 머신러닝은 학습을 반복적으로 수행
* 우리는 예측 변수, 모델 종류, 모델 파라미터 등을 선택해야함
* 지금까지는 validation data로 모델 성능을 측정하는 데이터 중심(data-driven) 방식으로 선택함
* 데이터 중심 방식의 단점은 validation data에서만 성능이 좋을 수 있다는 것임
    * 예를 들어 5000개의 데이터셋이 있을 때, 보통 20%(1000개)를 validation data로 사용함. 이 때 모델은 1000개의 데이터에서만 성능이 좋을 수 있음(== 다른 1000개에선 낮은 성능일 수 있음)
    * 극단적인 예로, validation data가 1개뿐이라면?
* 일반적으로 validation 데이터셋이 많을수록, 모델 성능 측정에 있어서 랜덤성(aka 'noise')이 적어지고, 신뢰성이 높아짐
* 그러나 validation 데이터셋이 많으려면 그만큼 학습 데이터셋을 줄여야 함
    * 학습 데이터셋이 적으면 좋지 않은 모델임
    
<br>

## What is cross-validation?

* cross-validation에서, 다수의 모델 성능 측정값을 얻기 위해 전체 데이터의 부분 집합들로 모델링을 수행함
* 예를 들어 전체 데이터를 5 그룹으로 나눌 수 있음(각각의 그룹은 전체 데이터의 20%). 이 때, 5 **fold**로 데이터를 나눴다고 말할 수 있음

<img src="../images/3/Dataset_5folds.png" width="100%">

* 각 fold에 대해 한 번의 실험(Experiment)을 수행
    * **Experiment 1**: 첫번째 fold를 validation(or holdout) set으로 사용하고, 나머지는 전부 training set으로 사용. 20% holdout set을 기반으로 측정된 모델 성능
    * **Experiment 2**: 두번째 fold를 제외한 모든 fold를 training set으로 사용. holdout set은 두번째 모델 성능 평가에 사용.
    * 위처럼 모든 fold를 holdout set으로 한 번씩 사용. 이 결과를 모두 모으면, 데이터셋은 100% 모두 사용해서 모델 성능을 평가(측정)하는 것이 됨
    
<br>

## When should you use cross-validation?

* cross-validation은 모델 성능을 더 정확하게 측정하게 해줌. 이것은 모델링 결정을 많이 할 때 특히 중요. but 각 fold 마다 성능을 측정해야해서 전체 모델링 시간이 더 길어질 수 있음
* 이러한 장단점을 고려할 때, cross-validation을 언제 사용해야할까?
    * *데이터셋이 작을 때*, 추가적인 계산에 대한 부담이 없으면 사용
    * *데이터셋이 클 때*, validation set 하나로도 충분. 데이터셋의 일부를 재사용해야할 필요가 없음
* 데이터셋이 크다, 작다의 기준은 없지만, 모델 학습에 걸리는 시간이 얼마 차이나지 않는다면 cross-validation을 사용하는 것이 좋음
* cross-validation을 했을 때, 각 실험의 결과가 비슷하면 하나의 validation set만 사용해도 충분하다고 봐도 됨

<br>

## Example

In [2]:
import pandas as pd

# Read the data
data = pd.read_csv('../input/melb_data.csv')

# Select subset of predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Select target
y = data.Price

* missing value를 imputation하고, random forest로 예측하기 위해 pipeline 정의
* pipeline없이 cross-validation을 할 수 있지만 매우 어려움!

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', RandomForestRegressor(n_estimators=50, random_state=0))
                             ])

* scikit-learn의 `cross_val_score()`로 cross-validtion score를 얻음. `cv` 파라미터로 fold의 수를 설정

In [8]:
from sklearn.model_selection import cross_val_score

## Multiply by -1 since sklearn calculates *negative* MAE ##
scores = -1 * cross_val_score(my_pipeline, X, y, cv=5, scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

MAE scores:
 [301628.7893587  303164.4782723  287298.331666   236061.84754543
 260383.45111427]


* `scroing`: 모델 성능을 평가하는 기준 설정([options](https://scikit-learn.org/stable/modules/model_evaluation.html))
* Scikit-learn은 관습적으로 모든 측정값이 높을수록 좋은 것으로 정의되어 있음. 그래서 이 관습을 따르려면, `neg_mean_absolute_error`를 사용하면 됨(MAE가 1000이면, NMAE는 -1000. error라는 것을 표현하기 위해 neg_mean_absolute_error를 사용하는 듯? 1000 그대로 사용하면 좋은 값같으니까). // negative MAE를 다른 곳에서는 거의 들어보지 못함

<br>
* cross-validation으로 모델 성능을 측정한 결과를 평균냄

In [10]:
print("Average MAE score (across experiments):")
print(scores.mean())

Average MAE score (across experiments):
277707.3795913405
